In [ ]:
!pip install nltk

In [1]:
from os import listdir
from nltk import ngrams
import hashlib

directories = ["Benign PE Samples", "Malicious PE Samples"]
N = 2

def read_file(file_path):
    with open(file_path, "rb") as binary_file:
        data = binary_file.read()
    return data

# 바이트 시퀸스에서 N-그램 리스트를 만든다.
def byte_sequence_to_Ngrams(byte_sequence, N):
    return ngrams(byte_sequence, N)

# 입력값의 해시값을 계산한다.
def hash_input(inp):
    return int(hashlib.md5(inp).hexdigest(), 16)

# N-그램을 바이트로 변환한다.
def make_ngram_hashable(Ngram):
    return bytes(Ngram)

# N-그램의 해시값을 계산한 다음, 해시값에 대한 딕셔너리의 카운트를 증가시킨다.
def hash_file_Ngrams_into_directory(file_Ngrams, T):
    for Ngram in file_Ngrams:
        hashable_Ngram = make_ngram_hashable(Ngram)
        hashed_and_reduced = hash_input(hashable_Ngram) % B # 축소모듈 B는 딕셔너리에 B 키값만 입력하도록 함.
        T[hashed_and_reduced] = T.get(hashed_and_reduced, 0) + 1

SyntaxError: invalid syntax (Temp/ipykernel_13856/1640213015.py, line 26)

In [ ]:
B = 65521 # 2^16보다 작으면서 가장 큰 소수를 B로 지정
T = {}

# 해시값을 계산한 N-그램의 갯수를 센다.
for dataset_path in directories:
    samples = [ f for f in listdir(dataset_path)]
    for file in samples:
        file_path = dataset_path + "/" + file
        file_byte_sequence = read_file(file_path)
        file_Ngrams = byte_sequence_to_Ngrams(file_byte_sequence, N)
        hash_file_Ngrams_into_directory(file_Ngrams, T)

In [ ]:
import heapq

# 가장 빈도수가 높은 K1=1000개를 선택
K1 = 1000
K1_most_common_Ngrams_Using_Hash_Grams = heapq.nlargest(K1, T)

# 가장 빈도수가 높은 N-그램의 해시값을 선택해 특성 집합으로 구성함
def featurize_sample(file,K1_most_common_Ngrams_Using_Hash_Grams):
    # 샘플의 특성 벡터를 만든다. 특성은 우리가 선택한 K1개의 N-그램의 갯수다.
    K1 = len(K1_most_common_Ngrams_Using_Hash_Grams)
    fv = K1 * [0]
    file_byte_sequence = read_file(file_path)
    file_Ngrams = byte_sequence_to_Ngrams(file_byte_sequence, N)
    for Ngram in file_Ngrams:
        hashable_Ngram = make_ngram_hashable(Ngram)
        hashed_and_reduced = hash_input(hashable_Ngram) % B
        if hashed_and_reduced in K1_most_common_Ngrams_Using_Hash_Grams:
            index = K1_most_common_Ngrams_Using_Hash_Grams.index(hashed_and_reduced)
            fv[index] += 1
    return fv

In [ ]:
# 데이터 셋을 특성화
X = []
for dataset_path in directories:
    samples = [f for f in listdir(dataset_path)]
    for file in samples:
        file_path = dataset_path + "/" + file
        X.append(featurize_sample(file_path, K1_most_common_Ngrams_Using_Hash_Grams))